In [ ]:
# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

## Example use

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [ ]:
def display_image(image):
    fig = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.imshow(image)
    #plt.savefig("test.png")
    #cv2.imshow("Frame", image)
    


# def download_and_resize_image(url, new_width=256, new_height=256,
#                               display=False):
#     _, filename = tempfile.mkstemp(suffix=".jpg")
#     response = urlopen(url)
#     image_data = response.read()
#     image_data = BytesIO(image_data)
#     pil_image = Image.open(image_data)
#     pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
#     pil_image_rgb = pil_image.convert("RGB")
#     pil_image_rgb.save(filename, format="JPEG", quality=90)
#     print("Image downloaded to %s." % filename)
#     if display:
#     display_image(pil_image)
#     return filename


def draw_bounding_box_on_image(image,  ymin, xmin, ymax, xmax, color, font, thickness=4, display_str_list=()):
    """Adds a bounding box to an image."""
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)
    
   # print(left, right, top , bottom)
    # If the total height of the display strings added to the top of the bounding
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str,
                  fill="black",
                  font=font)
        text_bottom -= text_height - 2 * margin


# def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
#     """Overlay labeled boxes on an image with formatted scores and label names."""
#     colors = list(ImageColor.colormap.values())

#     try:
#         font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
#                               25)
#     except IOError:
#         print("Font not found, using default font.")
#         font = ImageFont.load_default()

#     for i in range(min(boxes.shape[0], max_boxes)):
#         if scores[i] >= min_score:
#             ymin, xmin, ymax, xmax = tuple(boxes[i])
#             display_str = "{}: {}%".format(class_names[i].decode("ascii"),
#                                          int(100 * scores[i]))
#             color = colors[hash(class_names[i]) % len(colors)]
#             image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
#             draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color, font, display_str_list=[display_str])
#             np.copyto(image, np.array(image_pil))
#     return image

def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.3):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())

    font = ImageFont.load_default()

    for i in range(min(len(boxes), max_boxes)):
        #if scores[i] >= min_score:
        ymin, xmin, ymax, xmax = tuple(boxes[i])
        display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
        color = colors[hash(class_names[i]) % len(colors)]
        image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
        draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color, font, display_str_list=[display_str])
        np.copyto(image, np.array(image_pil))
    return image

## Apply module

Load a public image from Open Images v4, save locally, and display.

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [ ]:
module_handle = "C:\\Users\\soena" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

In [ ]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img

def run_detector(detector, path):
    img = load_img(path)

    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    start_time = time.time()
    result = detector(converted_img)
    #print(result)
    end_time = time.time()

    result = {key:value.numpy() for key,value in result.items()}
    
    #print("Found %d objects." % len(result["detection_scores"]))
    print("Inference time: ", end_time-start_time)
        
    boxes = []
    scores=[]
    entities = []
    for i in range(len(result["detection_boxes"])):

        if (result["detection_scores"][i]<0.2):
            break
        ymin, xmin, ymax, xmax = tuple(result["detection_boxes"][i])
        flag = 0
        for item in boxes:
            y1,x1,y2,x2 = tuple(item)
            total_diff = abs(ymin-y1)+abs(xmin-x1)+abs(ymax-y2)+abs(xmax -x2)
            if(total_diff < 0.1):
                flag=1
                break
        if(result["detection_class_entities"][i].decode("ascii") in ['Truck', 'Bus', 'Land vehicle' ,'Vehicle'] and i!=0):
            flag=1
        if(flag==0 or i==0):
            boxes.append(list(result["detection_boxes"][i]))
            scores.append(result["detection_scores"][i])
            entities.append(result["detection_class_entities"][i])
        
    image_with_boxes = draw_boxes(img.numpy(), boxes, entities, scores)
    for i in range(len(entities)):
        print(str(entities[i].decode("ascii")) + " ------ "+ str(int(scores[i]*100))+"%")
    green =[2]
    pink = [4,5,6]
    
    if(entities[0]==b'Truck' or entities[0]==b'Bus'):
        if(entities.count(b'Wheel')+entities.count(b'Tire') in green):
            print("FASTag Category - Green (2 Axle Trucks and Buses)")
        elif(entities.count(b'Wheel')+entities.count(b'Tire') == 3):
            print("FASTag Category - Yellow (3 Axle Trucks and Buses)")
        elif(entities.count(b'Wheel')+entities.count(b'Tire') in pink):
            print("FASTag Category - Pink (4, 5 or 6  Axle Trucks and Buses)")
        elif(entities.count(b'Wheel')+entities.count(b'Tire') > 6):
            print("FASTag Category - Sky Blue (7+  Axle Trucks and Buses)")
        else:
            print("FASTag Category - Green (2 or 3 Axle Trucks and Buses)")
    elif(entities[0]==b'Van'):
        print("FASTag Category - Orange (Light Commercial Vehicles)")
    elif(entities[0]==b'Car' or entities[0]==b'Jeep' or entities[0]==b'Taxi'):
        print("FASTag Category - Violet (Car, Jeep, Van, Taxi)")
    elif(entities[0]==b'Land vehicle'):
        print("FASTag Category - Black (Bulldozer and Earth-movers)")
    else:
        print("Unable to detect FASTag Category!")
    display_image(image_with_boxes)
    return result

In [ ]:
result = run_detector(detector, './data/4.jpg')

In [ ]:
type(result)

In [ ]:
ymin, xmin, ymax, xmax = tuple(result["detection_boxes"][i])

In [ ]:
def run_detector1(detector, img):
    img = load_img(img)
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    start_time = time.time()
    result = detector(converted_img)
    end_time = time.time()

    result = {key:value.numpy() for key,value in result.items()}

    print("Found %d objects." % len(result["detection_scores"]))
    print("Inference time: ", end_time-start_time)

    image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

    display_image(image_with_boxes)

In [ ]:
run_detector(detector, downloaded_image_path)

In [ ]:
run_detector1(detector, r"C:\Users\soena\videoplayback (1).mp4")

### More images
Perform inference on some additional images with time tracking.


In [ ]:
image_urls = [
  # Source: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
  "https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg",
  # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg",
  # Source: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
  "https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg",
  ]

def detect_img(image_url):
  start_time = time.time()
  #image_path = download_and_resize_image(image_url, 640, 480)
  run_detector(detector, image_url)
  end_time = time.time()
  print("Inference time:",end_time-start_time)

In [ ]:
detect_img(r"C:\Users\soena\dog.jpg")

In [ ]:
detect_img(image_urls[1])

In [ ]:
detect_img(image_urls[2])

In [ ]:
import cv2

In [ ]:
cam = cv2.VideoCapture(r"C:\Users\soena\test.mp4")
 

In [ ]:
currentframe = 0
while(True):
      
    # reading from frame
    ret,frame = cam.read()
  
    if ret:
        name = './data/frame' + str(currentframe) + '.jpg'
        print ('Creating...' + name)
  
        # writing the extracted images
        cv2.imwrite(name, frame)
  
        # increasing counter so that it will
        # show how many frames are created
        currentframe += 1
        #run_detector1(detector, name)
        # if video is still left continue creating images
        # writing the extracted images
          
        # increasing counter so that it will
        # show how many frames are created

    else:
        break
  
# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()

In [ ]:
run_detector(detector, './data/7.jpg')